In [2]:
import numpy as np
import pandas as pd
import random
from datetime import datetime

pd.options.display.float_format = '{:,.5f}'.format

#read in data
airportRegions = pd.read_csv('AirportRegions.csv')
paxDemand = pd.read_csv('PaxDemand.csv')
paxDemandDist = pd.read_csv('PaxDemandDist.csv')
ticketPrices = pd.read_csv('TicketPrices.csv')
zip2Regions = pd.read_csv('Zip2Regions.csv')

#1: number of airports per region
print(airportRegions.groupby('Region').size())

Region
Central       90
Mexico         1
Midwest      151
Northeast    219
Northwest     48
Southeast    141
Southwest     26
US-Non-48      1
Western      218
dtype: int64


In [75]:
#2 the region with the most zip2 values
print(zip2Regions.groupby('Region').size().idxmax())

Northeast


In [4]:
#3 expected weekly passenger arrivals, expected weekly revenue for each airport
paxDemandDistGrouped = paxDemandDist.groupby('OrigZip2')
ticketPricesGrouped = ticketPrices.groupby('OrigZip2')

###
flights = {'OrigZip2':[],
           'Airport': [],
           'Passengers': [],
           'Revenue': []}

#outer loop goes through paxDemand, stores zip2s and weeklyfc to FCdata
for index, FCdata in paxDemand.iterrows():
    
    #a is a group of demand distributions for a zip2
    a = paxDemandDistGrouped.get_group(FCdata['Zip2'])
    flights['Airport'].extend(list(a['DestAirport']))
    ###
    flights['OrigZip2'].extend(list(a['OrigZip2']))
    ###
    
    
    #b is a group of prices for a zip2
    b = ticketPricesGrouped.get_group(FCdata['Zip2'])
    
    
    #resize a and multiply weekly forecasted demand by the distributions
    a = a.loc[:,['FirstClass','BusinessClass','MainCabin','Economy']]*FCdata['WeeklyTicketFC']

    #resize b to just price data
    b = b.loc[:,['FirstClass','BusinessClass','MainCabin','Economy']]
    
   
    #element wise multiplication of demand*demanddist * price
    product = pd.DataFrame(a.values*b.values*.01, columns=a.columns)

    
    #sum arrivals over the classes in a, store in a new column of sums
    a['totalarrivals'] = a.apply(lambda x: x['FirstClass'] + x['BusinessClass'] + x['MainCabin'] + x['Economy'], axis=1)
    a['totalarrivals'] = a['totalarrivals']*.01
    
    #sum revenue per class in product, store in a new column of sums
    product['totalmoney'] = product.apply(lambda x: x['FirstClass'] + x['BusinessClass'] + x['MainCabin'] + x['Economy'], axis=1)
    
    
    flights['Passengers'].extend(list(a['totalarrivals']))
    flights['Revenue'].extend(list(product['totalmoney']))
    
Aggflights = pd.DataFrame(flights)
#print(Aggflights)
airportlist = Aggflights.Airport.unique()

report = {'Airport': [],
        'WeeklyArrivals': [],
        'WeeklyRevenue': []}

for airport in airportlist:
    report['Airport'].append(airport)
    yeet = Aggflights.loc[Aggflights.Airport==airport]
    report['WeeklyArrivals'].append(yeet['Passengers'].sum())
    report['WeeklyRevenue'].append(yeet['Revenue'].sum())

FinalReport = pd.DataFrame(report)
print(FinalReport)
# FinalReport.to_csv('number3.csv')

#CHANGE TO PRINT TO CSV
    


   Airport  WeeklyArrivals  WeeklyRevenue
0      ABE       861.19714  325,121.72330
1      BFI       733.47429  317,503.96395
2      BNA       796.07543  286,261.68557
3      BWI       751.82378  276,416.91150
4      CAE       840.94647  307,477.08888
5      CHA       920.84519  338,752.52403
6      DFW       915.59618  359,730.64354
7      EWR       927.52039  334,197.93536
8      IND       853.62283  302,100.98333
9      LEX       708.80896  261,946.15593
10     MKE       643.45225  244,638.05767
11     OAK       932.84819  386,992.53180
12     ONT       975.80851  417,723.92298
13     PHL       696.86180  264,276.63193
14     PHX       723.68098  298,047.57647
15     RIC       768.56939  271,650.09676
16     TPA       746.29838  287,710.51966


In [77]:
#4 expected weekly passenger arrivals and revenue per region
regionlist = airportRegions.Region.unique()
secondreport = {'Region': [],
                'WeeklyArrivals': [0,0,0,0,0,0,0,0,0],
                'WeeklyRevenue': [0,0,0,0,0,0,0,0,0]}
secondreport['Region'].extend(regionlist)
dfsecondreport = pd.DataFrame(secondreport)
dfsecondreport = dfsecondreport.set_index('Region')

for index, aggdata in Aggflights.iterrows():
    row = airportRegions.loc[airportRegions.Airport==aggdata['Airport']]
    regionvalue = row.Region.unique().item()
#     print(regionvalue)
#     break
    if regionvalue == 'Northeast':
        dfsecondreport.loc['Northeast','WeeklyArrivals'] = dfsecondreport.loc['Northeast','WeeklyArrivals'] + aggdata.Passengers
        dfsecondreport.loc['Northeast','WeeklyRevenue'] = dfsecondreport.loc['Northeast','WeeklyRevenue'] + aggdata.Revenue
    elif regionvalue == 'Central':
        dfsecondreport.loc['Central','WeeklyArrivals'] = dfsecondreport.loc['Central','WeeklyArrivals'] + aggdata.Passengers
        dfsecondreport.loc['Central','WeeklyRevenue'] = dfsecondreport.loc['Central','WeeklyRevenue'] + aggdata.Revenue
    elif regionvalue == 'Western':
        dfsecondreport.loc['Western','WeeklyArrivals'] = dfsecondreport.loc['Western','WeeklyArrivals'] + aggdata.Passengers
        dfsecondreport.loc['Western','WeeklyRevenue'] = dfsecondreport.loc['Western','WeeklyRevenue'] + aggdata.Revenue
    elif regionvalue == 'Southeast':
        dfsecondreport.loc['Southeast','WeeklyArrivals'] = dfsecondreport.loc['Southeast','WeeklyArrivals'] + aggdata.Passengers
        dfsecondreport.loc['Southeast','WeeklyRevenue'] = dfsecondreport.loc['Southeast','WeeklyRevenue'] + aggdata.Revenue
    elif regionvalue == 'Midwest':
        dfsecondreport.loc['Midwest','WeeklyArrivals'] = dfsecondreport.loc['Midwest','WeeklyArrivals'] + aggdata.Passengers
        dfsecondreport.loc['Midwest','WeeklyRevenue'] = dfsecondreport.loc['Midwest','WeeklyRevenue'] + aggdata.Revenue
    elif regionvalue == 'Northwest':
        dfsecondreport.loc['Northwest','WeeklyArrivals'] = dfsecondreport.loc['Northwest','WeeklyArrivals'] + aggdata.Passengers
        dfsecondreport.loc['Northwest','WeeklyRevenue'] = dfsecondreport.loc['Northwest','WeeklyRevenue'] + aggdata.Revenue
    elif regionvalue == 'Southwest':
        dfsecondreport.loc['Southwest','WeeklyArrivals'] = dfsecondreport.loc['Southwest','WeeklyArrivals'] + aggdata.Passengers
        dfsecondreport.loc['Southwest','WeeklyRevenue'] = dfsecondreport.loc['Southwest','WeeklyRevenue'] + aggdata.Revenue
    elif regionvalue == 'US-Non-48':
        dfsecondreport.loc['US-Non-48','WeeklyArrivals'] = dfsecondreport.loc['US-Non-48','WeeklyArrivals'] + aggdata.Passengers
        dfsecondreport.loc['US-Non-48','WeeklyRevenue'] = dfsecondreport.loc['US-Non-48','WeeklyRevenue'] + aggdata.Revenue
    else:
        dfsecondreport.loc['Mexico','WeeklyArrivals'] = dfsecondreport.loc['Mexico','WeeklyArrivals'] + aggdata.Passengers
        dfsecondreport.loc['Mexico','WeeklyRevenue'] = dfsecondreport.loc['Mexico','WeeklyRevenue'] + aggdata.Revenue

print(dfsecondreport)
dfsecondreport.to_csv('number4.csv')

#I know this code is terribly ugly and would not work if there were more regions 
#yikes
#Im not that eloquent with pandas yet

#CHANGE TO WRITE TO CSV

           WeeklyArrivals   WeeklyRevenue
Region                                   
Northeast     4,005.97250 1,471,663.29885
Central         915.59618   359,730.64354
Western       1,908.65669   804,716.45478
Southeast     3,304.16548 1,220,201.81814
Midwest       2,205.88404   808,685.19692
Northwest       733.47429   317,503.96395
Southwest       723.68098   298,047.57647
US-Non-48         0.00000         0.00000
Mexico            0.00000         0.00000


In [127]:
#5
#user enters a zip2. make a list of the airports you can go to from that zip2
#rng an airport from that list and remove that flight
#"we can no longer fly from zip2 xx to airport AAA"
#youre essentially deleting a flight
paxDemandDist = pd.read_csv('PaxDemandDist.csv')
paxDemandDistGrouped = paxDemandDist.groupby('OrigZip2')

flag = False
listofzips = paxDemandDist.OrigZip2.unique()
while flag == False:
    uzip2 = int(input("Enter a valid Zip2: "))
    for zip in listofzips:
        if str(uzip2) == str(zip):
            flag = True
            break
    

zipgroup = paxDemandDistGrouped.get_group(uzip2)

rngidx = random.choice(zipgroup.index.values)
#1, rngidx = 5, We can no longer fly from Zip2 1 to  CHA .
rngidx = 5
print("We can no longer fly from Zip2",uzip2,"to ",zipgroup.loc[rngidx,'DestAirport'],".")
zipgroup = zipgroup.drop(index = rngidx)
thesum = sum(zipgroup.loc[:,['FirstClass','BusinessClass','MainCabin','Economy']].sum())

normalize = lambda x: (x/thesum)*100

zipgroup[['FirstClass','BusinessClass','MainCabin','Economy']] = zipgroup[['FirstClass','BusinessClass','MainCabin','Economy']].applymap(normalize)
paxDemandDist = paxDemandDist.drop(index = rngidx)

for index, data in zipgroup.iterrows():
    #zipgroup.loc[index,['FirstClass','BusinessClass','MainCabin','Economy']]
    paxDemandDist.loc[index,['FirstClass','BusinessClass','MainCabin','Economy']] = zipgroup.loc[index,['FirstClass','BusinessClass','MainCabin','Economy']]

paxDemandDistGrouped = paxDemandDist.groupby('OrigZip2')
print(paxDemandDistGrouped.get_group(uzip2))
#this print proves it worked!

#WRITE PAX DEMAND DIST TO CSV

Enter a valid Zip2: 1
We can no longer fly from Zip2 1 to  CHA .
    OrigZip2 DestAirport  FirstClass  BusinessClass  MainCabin  Economy
0          1         ABE     0.08150        3.39887    2.47040  0.42183
1          1         BFI     0.19384        4.74255    4.43306  0.50664
2          1         BNA     0.00110        1.64657    1.05733  0.08371
3          1         BWI     0.00110        0.83264    1.00226  0.09802
4          1         CAE     0.03194        0.52866    0.32931  0.14538
6          1         DFW     0.30839        5.28113    4.99036  0.44716
7          1         EWR     0.09802        3.38455    2.96712  0.42403
8          1         IND     1.53312        8.65576   10.24616  0.78529
9          1         LEX     0.47249       13.06349    8.33967  0.82273
10         1         MKE     0.00110        1.75560    2.12236  0.37006
11         1         OAK     0.01652        0.81612    0.42844  0.06718
12         1         ONT     0.01652        1.26439    1.07825  0.18063

In [81]:
#6
paxDemand = pd.read_csv('PaxDemand.csv')
paxDemandDistGrouped = paxDemandDist.groupby('OrigZip2')
ticketPricesGrouped = ticketPrices.groupby('OrigZip2')

sixthreport = {'Zip2':[],
               'HistoricWeeklyFC':[],
               'NewMean':[],
               'NewSD':[],
               'WeeklyRevenueCI':[]}

for index, FCdata in paxDemand.iterrows():
    
    #a is a group of demand distributions for a zip2
    a = paxDemandDistGrouped.get_group(FCdata['Zip2'])
    
    #b is a group of prices for a zip2
    b = ticketPricesGrouped.get_group(FCdata['Zip2'])
    
    sixthreport['Zip2'].append(FCdata['Zip2'])
    sixthreport['HistoricWeeklyFC'].append(FCdata['WeeklyTicketFC'])
    
    historicmean = FCdata['WeeklyTicketFC']
    low = historicmean - (historicmean*.1)
    high = historicmean + (historicmean*.1)
    sample = np.random.uniform(low,high,100)
    
    newmean = np.mean(sample)
    newstd = np.std(sample)
    
    #because the data has more than 30 entries I will use the CLT to construct the CI
    CI = [newmean - 1.96*(newstd/10),newmean + 1.96*(newstd/10)]
    
    c = a.loc[:,['FirstClass','BusinessClass','MainCabin','Economy']]*CI[1]
    
    #resize a and multiply weekly forecasted demand by the distributions
    a = a.loc[:,['FirstClass','BusinessClass','MainCabin','Economy']]*CI[0]
  
    #resize b to just price data
    b = b.loc[:,['FirstClass','BusinessClass','MainCabin','Economy']]
    
   
    #element wise multiplication of demand*demanddist * price
    product = pd.DataFrame(a.values*b.values*.01, columns=a.columns)
    
    #sum over classes
    product['totalmoneylow'] = product.apply(lambda x: x['FirstClass'] + x['BusinessClass'] + x['MainCabin'] + x['Economy'], axis=1)
 
   #c = a.loc[:,['FirstClass','BusinessClass','MainCabin','Economy']]*CI[1]
    d = b.loc[:,['FirstClass','BusinessClass','MainCabin','Economy']]
    newproduct = pd.DataFrame(c.values*d.values*.01, columns=c.columns)
    product['totalmoneyhigh'] = newproduct.apply(lambda x: x['FirstClass'] + x['BusinessClass'] + x['MainCabin'] + x['Economy'], axis=1)
    
    thelowvalue = sum(product.loc[:,'totalmoneylow'])
    thelowvalue = float("{0:.2f}".format(thelowvalue))
    thehighvalue = sum(product.loc[:,'totalmoneyhigh'])
    thehighvalue = float("{0:.2f}".format(thehighvalue))
    
    revenueCI = [thelowvalue, thehighvalue]
    
    sixthreport['NewMean'].append(newmean)
    sixthreport['NewSD'].append(newstd)
    sixthreport['WeeklyRevenueCI'].append(revenueCI)

Sixthreport = pd.DataFrame(sixthreport)
Sixthreport.head()

#WRITE TO CSV
Sixthreport.to_csv('number6.csv',index=False)
    
    

In [82]:
#7 
paxDemand = pd.read_csv('PaxDemand.csv')
paxDemandDistGrouped = paxDemandDist.groupby('OrigZip2')

seventhreport = {'Orig Zip2':[],
               'Number of Airports':[],
               'First Class':[],
               'Business Class':[],
               'Main Cabin':[],
               'Economy':[]}

for index, FCdata in paxDemand.iterrows():
    
    #a is a group of demand distributions for a zip2
    a = paxDemandDistGrouped.get_group(FCdata['Zip2'])
    seventhreport['Orig Zip2'].append(FCdata['Zip2'])
    seventhreport['Number of Airports'].append(len(a['DestAirport']))
    seventhreport['First Class'].append(np.mean(a['FirstClass']))
    seventhreport['Business Class'].append(np.mean(a['BusinessClass']))
    seventhreport['Main Cabin'].append(np.mean(a['MainCabin']))
    seventhreport['Economy'].append(np.mean(a['Economy']))


Seventhreport = pd.DataFrame(seventhreport)
Seventhreport
#WRITE TO CSV??

,Orig Zip2,Number of Airports,First Class,Business Class,Main Cabin,Economy
0,1.00000,17,0.15759,2.88894,2.54353,0.29247
1,2.00000,17,0.20018,2.86906,2.53000,0.28318
2,3.00000,17,0.06165,3.25788,2.26918,0.29365
3,4.00000,17,0.17429,3.07829,2.37306,0.25688
4,5.00000,17,0.13147,2.99282,2.53382,0.22424
5,6.00000,17,0.36024,2.31765,2.78406,0.42059
6,7.00000,17,0.51559,0.32629,4.01212,1.02859
7,8.00000,17,0.37318,0.40753,4.04400,1.05765
8,10.00000,17,0.21329,0.37700,4.08094,1.21100
9,11.00000,17,0.35482,0.33606,4.16276,1.02859


In [80]:
copy = ticketPrices[:]

#copy['Date'] = pd.to_datetime(copy['Date'],infer_datetime_format=True) 


copy['Date'] = pd.DatetimeIndex(copy['Date'])
copy = copy.set_index('Date')
#QUARTERS WHERE THE YEAR ENDS IN DECEMBER!
report8 = copy.resample('Q-DEC',kind='timestamp').mean()
report8 = report8.drop('OrigZip2',axis = 1)
report8.head(20)
report8.to_csv('number8.csv')
